# Cross-resonance gate between two transmons: leakage

In [23]:
import numpy as np
import scipy.integrate
import time
import qutip as qtp
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pysqkit
from pysqkit import QubitSystem, Qubit
from pysqkit.drives.pulse_shapes import gaussian_top
from pysqkit.util.metrics import average_process_fidelity, \
    average_gate_fidelity
from pysqkit.util.phys import temperature_to_thermalenergy
from pysqkit.util.quantum import generalized_rabi_frequency
import pysqkit.util.transformations as trf
from pysqkit.util.linalg import get_mat_elem
from pysqkit.solvers.solvkit import integrate
from pysqkit.util.hsbasis import weyl_by_index
from pysqkit.solvers import solvkit
from pysqkit.drives.pulse_shapes import gaussian_top
import qutip
from typing import List, Dict, Callable
import multiprocessing
import util_cr as util
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import copy
import json
import cmath

from IPython.display import display, Latex

In [24]:
def mu_yz_c(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'])
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 - yz1))/2

def mu_yi_c(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'] )
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 + yz1))/2

def cr_effect_sw(
    jc: float,
    eps_drive: float,
    transm_t: Qubit,
    transm_c: Qubit
) -> float:
    """
    Description
    ---------------------------------------------------------------------
    Returns the cross-resonance coefficient in the linear approximation
    using the result of Magesan-Gambetta
    """
    j_eff = jc*transm_c.charge_zpf*transm_t.charge_zpf
    delta_t = transm_t.anharm
    delta_c = transm_c.anharm
    detuning = transm_c.freq - transm_t.freq
    return j_eff*transm_c.charge_zpf*eps_drive/detuning*(delta_c/(delta_c + detuning))/2

def func_to_minimize(
    pulse_time: list,
    t_rise: float,
    cr_coeff: float
) -> float:
    step = 1e-3
    n_points = int(pulse_time[0]/step)
    times = np.linspace(0, pulse_time[0], n_points)
    pulse = gaussian_top(times, t_rise, pulse_time[0])
    integral = scipy.integrate.simpson(2*np.pi*cr_coeff*pulse, times)
    return np.abs(integral - np.pi/4)   

In [21]:
def get_leakage(
    transm_freq: float
) -> dict:
    with open('../flx_transm_params.txt') as param_file:
        parameters_set = json.load(param_file)
    temperature = 0.020 #0.020 # K
    thermal_energy = temperature_to_thermalenergy(temperature) # kb T/h in GHz
    d_comp = 4

    control_freq = 5.0

    #Transmon
    levels_t = 3
    transm_t = pysqkit.qubits.SimpleTransmon(
        label='T', 
        max_freq=transm_freq, 
        anharm=-0.3,
        diel_loss_tan=0.3*1e-6, #set to zero to check d_1 L1 = d_2 L2
        env_thermal_energy=thermal_energy,    
        dim_hilbert=levels_t,
        dephasing_times=None #parameters_set[p_set]["dephasing_times_t"]
    )

    transm_c = pysqkit.qubits.SimpleTransmon(
        label='C', 
        max_freq=control_freq, 
        anharm=-0.3,
        diel_loss_tan=0.3*1e-6, #set to zero to check d_1 L1 = d_2 L2
        env_thermal_energy=thermal_energy,    
        dim_hilbert=levels_t,
        dephasing_times=None #parameters_set[p_set]["dephasing_times_t"]
    )

    # We also add a drive on the fluxonium
    transm_c.add_drive(
        pysqkit.drives.microwave_drive,
        label='cr_drive',
        pulse=pysqkit.drives.pulses.cos_modulation,
        pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
    )

    d_leak = levels_t**2 - d_comp

    jc = 0.002
    coupled_sys = transm_t.couple_to(transm_c, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)


    states_label = coupled_sys.all_state_labels()
    states_dict = coupled_sys.states_as_dict(as_qobj=True)
    
    state_label = ["00", "01", "10", "11"]
    comp_states = {}
    for label in state_label:
        state_tmp = coupled_sys.state(label)[1]
        loc = np.argmax(np.abs(state_tmp))
        phase = cmath.phase(state_tmp[loc])
        state_tmp = np.exp(-1j*phase)*state_tmp
        comp_states[label] = state_tmp
    
    eps = 0.03
    omega_flx, states_flx = flx.eig_states(4)
    op = coupled_sys["F"].charge_op()
    freq_drive = transm.max_freq
    omega_drive = np.abs(get_mat_elem(op, coupled_sys.state("01")[1], coupled_sys.state("11")[1]))
    delta_drive = freq_drive - transm.max_freq
    rabi_period = 1/np.sqrt(omega_drive**2 + delta_drive**2)
    t_rise = 10.0 # [ns]

    t_gate_0 = [200.0]

    args_to_pass = (t_rise, np.abs(mu_yz_flx_sw(transm, flx, jc))*eps/2) 
    #args_to_pass = (t_rise, np.abs(mu_yz_flx(comp_states, op))*eps/2) 

    # We find the total time to obtain the desired gate

    start = time.time()

    minimization_result = minimize(func_to_minimize, t_gate_0, args=args_to_pass)

    print(minimization_result)

    end = time.time()

    t_gate = minimization_result['x'][0] #1/(util.y_z_flx(coupled_sys, 'F')*eps_drive*4)  # [ns]
    print("t_gate: {} ns".format(t_gate))
    pts_per_drive_period = 10

    #t_tot = 135

    nb_points = int(t_gate*freq_drive*pts_per_drive_period)
    tlist = np.linspace(0, t_gate, nb_points)

    coupled_sys['F'].drives['cr_drive_f'].set_params(phase=0, time=tlist, rise_time=t_rise, pulse_time=t_gate,
                                                     amp=eps, freq=freq_drive)

    simu_opt = qtp.solver.Options()
    simu_opt.atol = 1e-12
    simu_opt.rtol = 1e-10

    env_syst = pysqkit.tomography.TomoEnv(system=coupled_sys, time=2*np.pi*tlist, 
                                          options=simu_opt, with_noise=False)
    
    env_syst_noisy = pysqkit.tomography.TomoEnv(system=coupled_sys, time=2*np.pi*tlist, 
                                          options=simu_opt, with_noise=True)
    
    comp_states_list = []
    for key in comp_states.keys():
        comp_states_list.append(comp_states[key])
    
    avg_leakage = env_syst.leakage(comp_states_list)
    #avg_leakage_noisy = env_syst_noisy.leakage(comp_states_list)
    
#     sup_op = env_syst.to_super(comp_states_list, weyl_by_index)
#     sup_op_noisy = env_syst_noisy.to_super(comp_states_list, weyl_by_index)
    
#     cr_super_target = trf.kraus_to_super(cry(-np.pi/2), weyl_by_index)
    
#     opt_sup_op = optimal_sup_op(cr_super_target, sup_op)
#     opt_sup_op_noisy = optimal_sup_op(cr_super_target, sup_op_noisy)
    
#     f_gate = average_gate_fidelity(cr_super_target, opt_sup_op, avg_leakage)
#     f_gate_noisy = average_gate_fidelity(cr_super_target, opt_sup_op_noisy, avg_leakage_noisy)
    
    res={}
    res["transm_freq"] = transm_freq
    res["gate_time"] = t_gate
    res["eps"] = eps
    res["L1"] = avg_leakage
    
    #res["L1_noisy"] = avg_leakage_noisy
    #res["fidelity"] = f_gate
    #res["fidelity_noisy"] = f_gate_noisy
    
    return res
    

In [22]:
get_fidelity_leakage(4.405)

      fun: 1.653466252804492e-11
 hess_inv: array([[5507.42358234]])
      jac: array([0.0013992])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 96
      nit: 1
     njev: 42
   status: 2
  success: False
        x: array([227.63413589])
t_gate: 227.63413588553826 ns


{'transm_freq': 4.405,
 'gate_time': 227.63413588553826,
 'eps': 0.3,
 'L1': 0.05844844541667793}

In [5]:
n_points = 2
n_processes = 2
freq_list = np.linspace(4.2, 5.8, n_points)

start = time.time()

pool = multiprocessing.Pool(processes=n_processes)

result = pool.map(get_fidelity_leakage, freq_list, chunksize=int(n_points//n_processes))

pool.close()
pool.join()

end=time.time()

print("Computation time: {} s".format(end - start))

      fun: 9.028863434679124e-09
 hess_inv: array([[0.02852856]])
      jac: array([0.0232648])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 60
      nit: 1
     njev: 24
   status: 2
  success: False
        x: array([0.63589929])
      fun: 5.166372218923243e-09
 hess_inv: array([[0.15221642]])
      jac: array([0.89500821])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 84
      nit: 1
     njev: 36
   status: 2
  success: False
        x: array([0.49445029])


NameError: name 'start' is not defined

In [ ]:
save = True
if save:
    with open("/tmp/cr_fidelity_leakage.txt", "w") as fp:
        json.dump(result, fp)